<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/query_engine/knowledge_graph_query_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Knowledge Graph Query Engine

Creating a Knowledge Graph usually involves specialized and complex tasks. However, by utilizing the Llama Index (LLM), the KnowledgeGraphIndex, and the GraphStore, we can facilitate the creation of a relatively effective Knowledge Graph from any data source supported by [Llama Hub](https://llamahub.ai/).

Furthermore, querying a Knowledge Graph often requires domain-specific knowledge related to the storage system, such as Cypher. But, with the assistance of the LLM and the LlamaIndex KnowledgeGraphQueryEngine, this can be accomplished using Natural Language!

In this demonstration, we will guide you through the steps to:

- Extract and Set Up a Knowledge Graph using the Llama Index
- Query a Knowledge Graph using Cypher
- Query a Knowledge Graph using Natural Language

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [17]:
# %pip install llama-index-readers-wikipedia
# %pip install llama-index-llms-azure-openai
# %pip install llama-index-graph-stores-nebula
# %pip install llama-index-llms-openai
# %pip install llama-index-embeddings-azure-openai

Let's first get ready for basic preparation of Llama Index.

### OpenAI

In [18]:
# For OpenAI

import os

os.environ["OPENAI_API_KEY"] = "sk-a0khYUHBaQOMhZkcr5pNgNEcl6i9-4cmpmjKxIShyLT3BlbkFJxt6vnTi02sQ7JSk_39KIv9j4JoRS8tCO3mZ-3V8cQA"

import logging
import sys

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output


# define LLM
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

Settings.llm = OpenAI(temperature=0, model="gpt-4o-mini")
Settings.chunk_size = 512
# Settings.chunk_overlap =32

### Azure

In [19]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

# For OpenAI
api_key = "sk-a0khYUHBaQOMhZkcr5pNgNEcl6i9-4cmpmjKxIShyLT3BlbkFJxt6vnTi02sQ7JSk_39KIv9j4JoRS8tCO3mZ-3V8cQA"

llm = OpenAI(model="gpt-4o-mini")

# You need to deploy your own embedding model as well as your own chat completion model
embed_model = OpenAIEmbedding(model="text-embedding-3-large")


In [20]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model
# Settings.chunk_size = 512
# Settings.chunk_overlap =32

## Prepare for NebulaGraph

Before next step to creating the Knowledge Graph, let's ensure we have a running NebulaGraph with defined data schema.

In [21]:
%pip install neo4j

username = "neo4j"
password = "minhminh"
url = "bolt://localhost:7687"
database = "neo4j"

Note: you may need to restart the kernel to use updated packages.


Prepare for StorageContext with graph_store as NebulaGraphStore

In [22]:
from llama_index.graph_stores.neo4j import Neo4jGraphStore

graph_store = Neo4jGraphStore(
    username=username,
    password=password,
    url=url,
    database=database,
)

storage_context = StorageContext.from_defaults(graph_store=graph_store)

# NOTE: can take a while!
index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    max_triplets_per_chunk=2,
)

ValueError: Could not use APOC procedures. Please ensure the APOC plugin is installed in Neo4j and that 'apoc.meta.data()' is allowed in Neo4j configuration 

## (Optional)Build the Knowledge Graph with LlamaIndex

With the help of Llama Index and LLM defined, we could build Knowledge Graph from given documents.

If we have a Knowledge Graph on NebulaGraphStore already, this step could be skipped

### Step 1, load data from Wikipedia for "Guardians of the Galaxy Vol. 3"

In [11]:
# from llama_index.core import download_loader

# from llama_index.readers.wikipedia import WikipediaReader

# loader = WikipediaReader()

# documents = loader.load_data(
#     pages=["Guardians of the Galaxy Vol. 3"], auto_suggest=False
# )

In [12]:
from llama_index.core import SimpleDirectoryReader

reader = SimpleDirectoryReader(input_dir= r"C:\Users\My_Pc\Desktop\sample_test")
documents = reader.load_data(num_workers=4)

In [13]:
from llama_index.core import KnowledgeGraphIndex

# Tạo KnowledgeGraphIndex với các cạnh đã dịch và hai chiều
kg_index = KnowledgeGraphIndex.from_documents(
    documents=documents,
    storage_context=storage_context,
    max_triplets_per_chunk=10,
    space_name=space_name,
    edge_types=edge_types,  # Sử dụng cạnh hai chiều
    rel_prop_names=rel_prop_names,  # Sử dụng quan hệ đã dịch
    tags=tags,
    include_embeddings=True,
)

# In kết quả đồ thị để kiểm tra
print(kg_index)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


KeyboardInterrupt: 

In [29]:
query_engine = index.as_query_engine()

response = query_engine.query("Điều kiện hưởng lương hưu được quy định như thế nào?")

NameError: name 'index' is not defined

Now we have a Knowledge Graph on NebulaGraph cluster under space named `llamaindex` about the 'Guardians of the Galaxy Vol. 3' movie, let's play with it a little bit.

In [151]:
# install related packages, password is nebula by default
%pip install ipython-ngql networkx pyvis
%load_ext ngql
%ngql --address 127.0.0.1 --port 9669 --user root --password <nebula>

Note: you may need to restart the kernel to use updated packages.
The ngql extension is already loaded. To reload it, use:
  %reload_ext ngql
[OK] Connection Pool Created
INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


,Name
0,quocminhdb
1,test_data


In [152]:
# Query some random Relationships with Cypher
%ngql USE tvpl_graph;
%ngql MATCH ()-[e]->() RETURN e LIMIT 10

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)
INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


,e
0,"(""Luật bảo hiểm xã hội 2014"")-[:relationship@6148495660377499677{relationship: ""Có hiệu lực thi hành từ""}]->(""Ngày 01 tháng 01 năm 2018"")"
1,"(""Mức lương hưu"")-[:relationship@6173169967880352367{relationship: ""Được tính bằng""}]->(""45 % mức bình quân tiền lương"")"


## Asking the Knowledge Graph

Finally, let's demo how to Query Knowledge Graph with Natural language!

Here, we will leverage the `KnowledgeGraphQueryEngine`, with `NebulaGraphStore` as the `storage_context.graph_store`.

In [154]:
graph_store = NebulaGraphStore(
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
)

storage_context = StorageContext.from_defaults(graph_store=graph_store)

# NOTE: can take a while!
index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    max_triplets_per_chunk=10,
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [155]:
query_engine = index.as_query_engine()

response = query_engine.query("Mức lương hưu hằng tháng được tính như thế nào")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.indices.knowledge_graph.retrievers:> No relationships found, returning nodes found by keywords.
INFO:llama_index.core.indices.knowledge_graph.retrievers:> No nodes found by keywords, returning empty response.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [156]:
# NOTE: can take a while!
new_index = KnowledgeGraphIndex.from_documents(
    documents,
    max_triplets_per_chunk=10,
    include_embeddings=True,
)

# query using top 3 triplets plus keywords (duplicate triplets are removed)
query_engine = index.as_query_engine(
    include_text=True,
    response_mode="tree_summarize",
    embedding_mode="hybrid",
    similarity_top_k=5,
)
response = query_engine.query(
    "Luật bảo hiểm xã hội 2014 quy định điều gì",
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.indices.knowledge_graph.retrievers:> No relationships found, returning nodes found by keywords.
INFO:llama_index.core.indices.knowledge_graph.retrievers:> No nodes found by keywords, returning empty response.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [157]:
graph_query = query_engine.generate_query(
    "Tell me about Peter Quill?",
)

graph_query = graph_query.replace("WHERE", "\n  WHERE").replace(
    "RETURN", "\nRETURN"
)

display(
    Markdown(
        f"""
```cypher
{graph_query}
```
"""
    )
)

AttributeError: 'RetrieverQueryEngine' object has no attribute 'generate_query'

We could see it helps generate the Graph query:

```cypher
MATCH (p:`entity`)-[:relationship]->(e:`entity`)
  WHERE p.`entity`.`name` == 'Peter Quill'
RETURN e.`entity`.`name`;
```
And synthese the question based on its result:

```json
{'e2.entity.name': ['grandfather', 'alternate version of Gamora', 'Guardians of the Galaxy']}
```

Of course we still could query it, too! And this query engine could be our best Graph Query Language learning bot, then :).

In [ ]:
%%ngql
MATCH (p:`entity`)-[e:relationship]->(m:`entity`)
  WHERE p.`entity`.`name` == 'Peter Quill'
RETURN p.`entity`.`name`, e.relationship, m.`entity`.`name`;

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


,p.entity.name,e.relationship,m.entity.name
0,Peter Quill,would return to the MCU,May 2021
1,Peter Quill,was abducted from Earth,as a child
2,Peter Quill,is leader of,Guardians of the Galaxy
3,Peter Quill,was raised by,a group of alien thieves and smugglers
4,Peter Quill,is half-human,half-Celestial


And change the query to be rendered

In [ ]:
%%ngql
MATCH (p:`entity`)-[e:relationship]->(m:`entity`)
  WHERE p.`entity`.`name` == 'Peter Quill'
RETURN p, e, m;

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


,p,e,m
0,"(""Peter Quill"" :entity{name: ""Peter Quill""})","(""Peter Quill"")-[:relationship@-84437522554765...","(""May 2021"" :entity{name: ""May 2021""})"
1,"(""Peter Quill"" :entity{name: ""Peter Quill""})","(""Peter Quill"")-[:relationship@-11770408155938...","(""as a child"" :entity{name: ""as a child""})"
2,"(""Peter Quill"" :entity{name: ""Peter Quill""})","(""Peter Quill"")-[:relationship@-79394488349732...","(""Guardians of the Galaxy"" :entity{name: ""Guar..."
3,"(""Peter Quill"" :entity{name: ""Peter Quill""})","(""Peter Quill"")-[:relationship@325695233021653...","(""a group of alien thieves and smugglers"" :ent..."
4,"(""Peter Quill"" :entity{name: ""Peter Quill""})","(""Peter Quill"")-[:relationship@555553046209276...","(""half-Celestial"" :entity{name: ""half-Celestia..."


In [ ]:
%ng_draw

nebulagraph_draw.html


The results of this knowledge-fetching query could not be more clear from the renderred graph then.